In [8]:
import funcs.data_wrangling as dw
import pandas as pd
import numpy as np
import pyarrow as pa
from dtype_diet import report_on_dataframe, optimize_dtypes
# import seaborn as sns
# import matplotlib.pyplot as plt
# import plotly.express as px
# from matplotlib import rcParams
# from statsmodels.tsa.seasonal import STL
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsforecast import StatsForecast
# from statsforecast.models import MSTL, AutoARIMA

#rcParams['figure.figsize'] = 15, 5

import warnings
warnings.filterwarnings('ignore')

In [5]:
data = dw.ons_data(freq='h', ano_inicio=2000, ano_fim=2023, idreg="S")
df = dw.pipeline(data, update=False)

In [14]:
proposed_df = report_on_dataframe(df.reset_index(), unit="MB")
new_df = optimize_dtypes(df.reset_index(), proposed_df)
print(f'Original df memory: {df.reset_index().memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Propsed df memory: {new_df.memory_usage(deep=True).sum()/1024/1024} MB')

Original df memory: 25.94708251953125 MB
Propsed df memory: 3.4858531951904297 MB


In [7]:
my_schema = pa.Schema.from_pandas(df)
my_schema

id_reg: string
desc_reg: string
load_mwmed: double
date: timestamp[ns]
-- schema metadata --
pandas: '{"index_columns": ["date"], "column_indexes": [{"name": null, "f' + 665

In [18]:
df_novo = df.reset_index().astype({
        "date": "datetime64[ns]",
        "id_reg": "category",
        "desc_reg": "category",
        "load_mwmed": "float64"
        })

In [21]:
my_schema = pa.Schema.from_pandas(df_novo)
my_schema

date: timestamp[ns]
id_reg: dictionary<values=string, indices=int8, ordered=0>
desc_reg: dictionary<values=string, indices=int8, ordered=0>
load_mwmed: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 803

In [22]:
%%time
df_novo.to_parquet('applications_processed.parquet')


CPU times: total: 62.5 ms
Wall time: 112 ms


In [24]:
%%time
df_novo.to_parquet('applications_processed.parquet', schema = my_schema)

CPU times: total: 62.5 ms
Wall time: 61.8 ms


In [33]:
%%time
df_t = pd.read_csv("../../../data/daily_load.csv", sep=";")

CPU times: total: 31.2 ms
Wall time: 31.9 ms


In [34]:
df_t.dtypes

date          object
id_reg        object
desc_reg      object
load_mwmed    object
dtype: object

In [29]:
%%time
df_t = pd.read_parquet('applications_processed.parquet')

CPU times: total: 46.9 ms
Wall time: 17 ms


In [30]:
df_t.dtypes

date          datetime64[ns]
id_reg              category
desc_reg            category
load_mwmed           float64
dtype: object